In [42]:
# import os.path
# import re
# import pprint as pp
import yaml
import pathlib

import bson
# import seaborn
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from matplotlib.backends.backend_pdf import PdfPages
# import scipy.signal
from pymongo import MongoClient
# from gridfs import GridFS

# import eim

# %matplotlib inline
# %load_ext autoreload
# %autoreload 2

# Setup connection to the database

In [6]:
creds_path = pathlib.Path('/home/paperspace/eim-creds.yml')
with open(creds_path, 'r') as stream:
    try:
        creds = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [7]:
client = MongoClient()
client = MongoClient(host='eim-data.musicsensorsemotion.com', 
                     port=27017, 
                     username=creds['username'], 
                     password=creds['password'], 
                     authSource='admin', 
                     ssl=True)
db = client.eim

In [69]:
all_trials = db.trials.find()

In [70]:
def get_trial_dict(trial, media_idx):
    media = db.media.find_one({'_id': bson.ObjectId(trial['media'][media_idx])})
    metadata = trial['metadata']
    return {
        'location': metadata['location'],
        'date': trial['date'].strftime('%d-%b-%Y'),
        'terminal': metadata['terminal'],
        'session': metadata['session_number'],
        'media_label': media['label'],
        'trial_id': str(trial['_id']),
        'signal_id': str(trial['signals'][media_idx])
    }

In [71]:
trial_dicts = []

for t in all_trials:
    for i in range(len(t['media'])):
        try:
            trial_dicts.append(get_trial_dict(t, i))
        except:
            pass

In [72]:
trials_df = pd.DataFrame(trial_dicts)

In [73]:
trials_df.shape

(57446, 7)

In [74]:
trials_df.to_csv('./all_trials_metadata.csv')